## Import libraries

In [1]:
from pymongo import MongoClient
import pandas as pd
import numpy as np

## Set office preferences

**1.** Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design --> ***companies***

**2.** Developers like to be near successful tech startups that have raised at least 1 Million dollars --> ***companies***

**3.** The CEO is vegan --> ***foursquare***

**4.** 30% of the company staff have at least 1 child --> ***foursquare***

**5.** Executives like Starbucks A LOT. Ensure there's a starbucks not too far --> ***foursquare***

**6.** Account managers need to travel a lot --> ***foursquare***

**7.** Everyone in the company is between 25 and 40, give them some place to go party --> ***foursquare***

## Connect MongoDB database with Python

In [2]:
client = MongoClient("localhost:27017")

In [3]:
db = client["Ironhack"]

In [4]:
c = db.get_collection("companies")

In [5]:
c.find_one()

{'_id': ObjectId('52cdef7c4bab8bd675297d8a'),
 'name': 'Wetpaint',
 'permalink': 'abc2',
 'crunchbase_url': 'http://www.crunchbase.com/company/wetpaint',
 'homepage_url': 'http://wetpaint-inc.com',
 'blog_url': 'http://digitalquarters.net/',
 'blog_feed_url': 'http://digitalquarters.net/feed/',
 'twitter_username': 'BachelrWetpaint',
 'category_code': 'web',
 'number_of_employees': 47,
 'founded_year': 2005,
 'founded_month': 10,
 'founded_day': 17,
 'deadpooled_year': 1,
 'tag_list': 'wiki, seattle, elowitz, media-industry, media-platform, social-distribution-system',
 'alias_list': '',
 'email_address': 'info@wetpaint.com',
 'phone_number': '206.859.6300',
 'description': 'Technology Platform Company',
 'created_at': datetime.datetime(2007, 5, 25, 6, 51, 27),
 'updated_at': 'Sun Dec 08 07:15:44 UTC 2013',
 'overview': '<p>Wetpaint is a technology platform company that uses its proprietary state-of-the-art technology and expertise in social media to build and monetize audiences for di

## Query: Companies that matches preferences - MONGO DB

- **Condition 1.** Designers like to go to design talks and share knowledge. There must be some nearby companies that also do design

- **Condition 2.** Developers like to be near successful tech startups that have raised at least 1 Million dollars

In [6]:
condition_1 = {"total_money_raised": {"$regex": "M$"}}
condition_2 = {"tag_list": {"$regex": "design"}}
query = {"$and": [condition_1, condition_2]}
projection = {"name":1, "offices":1, "total_money_raised": 1, "_id":0}
result = list(c.find(query, projection))
result

[{'name': 'Digg',
  'total_money_raised': '$45M',
  'offices': [{'description': None,
    'address1': '135 Mississippi St',
    'address2': '',
    'zip_code': '94107',
    'city': 'San Francisco',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 37.764726,
    'longitude': -122.394523}]},
 {'name': 'GENWI',
  'total_money_raised': '$7.1M',
  'offices': [{'description': '',
    'address1': '4966 El Camino Real',
    'address2': 'Suite 200',
    'zip_code': '94022',
    'city': 'Los Altos',
    'state_code': 'CA',
    'country_code': 'USA',
    'latitude': 33.8171,
    'longitude': -111.9035}]},
 {'name': 'Aviary',
  'total_money_raised': '$19M',
  'offices': [{'description': 'NYC',
    'address1': '243 West 30th Street',
    'address2': '11th Floor',
    'zip_code': '10001',
    'city': 'NY',
    'state_code': 'NY',
    'country_code': 'USA',
    'latitude': None,
    'longitude': None}]},
 {'name': 'Wix',
  'total_money_raised': '$58.5M',
  'offices': [{'description'

In [7]:
df = pd.DataFrame(result)
df.head()

,name,total_money_raised,offices
0,Digg,$45M,"[{'description': None, 'address1': '135 Missis..."
1,GENWI,$7.1M,"[{'description': '', 'address1': '4966 El Cami..."
2,Aviary,$19M,"[{'description': 'NYC', 'address1': '243 West ..."
3,Wix,$58.5M,"[{'description': '', 'address1': '', 'address2..."
4,Gilt Groupe,$236M,"[{'description': 'New York Office', 'address1'..."


In [8]:
df = df.explode('offices')
df = df.reset_index(drop=True)

In [9]:
df['offices_country_code'] = df['offices'].apply(lambda x: x['country_code'])

In [10]:
value_counts_offices_country = df['offices_country_code'].value_counts().to_dict()
value_counts_offices_country_tup = list(value_counts_offices_country.items())

[('USA', 28),
 ('GBR', 3),
 ('ISR', 2),
 ('DEU', 2),
 ('CAN', 1),
 ('AUS', 1),
 ('IND', 1),
 ('ROM', 1),
 ('ARG', 1)]

In [11]:
df = df[df['offices_country_code'].apply(lambda x: value_counts_offices_country_tup[0][0] in x)]
df = df.reset_index(drop=True)

In [12]:
df['offices_state_code'] = df['offices'].apply(lambda x : x['state_code'])

In [13]:
value_counts_offices_state_code = df['offices_state_code'].value_counts()
value_counts_offices_state_code

offices_state_code
CA    15
NY     5
FL     3
IL     2
WA     1
DC     1
OR     1
Name: count, dtype: int64

In [14]:
df = df[df['offices_state_code'].apply(lambda x: any(val in x for val in ['CA','NY', 'FL', 'IL']))]
df = df.reset_index(drop=True)

In [15]:
df['offices'][0]

{'description': None,
 'address1': '135 Mississippi St',
 'address2': '',
 'zip_code': '94107',
 'city': 'San Francisco',
 'state_code': 'CA',
 'country_code': 'USA',
 'latitude': 37.764726,
 'longitude': -122.394523}

In [16]:
df['offices_latitude'] = df['offices'].apply(lambda x : x['latitude'])
df['offices_longitude'] = df['offices'].apply(lambda x : x['longitude'])
df['offices_address_1'] = df['offices'].apply(lambda x : x['address1'])
df['offices_address_2'] = df['offices'].apply(lambda x : x['address2'])
df['offices_zip_code'] = df['offices'].apply(lambda x : x['zip_code'])

In [17]:
df = df.drop('offices', axis=1)

In [18]:
df = df.replace('', np.nan)

In [19]:
df.dropna(subset=['offices_latitude', 'offices_longitude', 'offices_address_1', 'offices_address_2'], how='all', inplace=True)
df = df.reset_index(drop=True)

In [20]:
df_subset = df.loc[df['offices_latitude'].isnull(), ['name', 'offices_address_1', 'offices_address_2']]
df_subset

,name,offices_address_1,offices_address_2
2,Aviary,243 West 30th Street,11th Floor
9,Squarespace,459 Broadway,5th Floor
14,Banyan Branch,"455 Broadway, 3rd Floor",NaN
15,Banyan Branch,1671 Appian Way,NaN
16,Netbiscuits,7th Floor,275 Seventh Ave
19,Gnodal,2225 East Bayshore Rd,Suite 200
20,Talenthouse,542 High St,NaN
21,Talenthouse,8810 Melrose Ave,NaN


In [21]:
NaN_name_list = df.loc[df['offices_latitude'].isnull(), 'name'].tolist()
NaN_name_list

['Aviary',
 'Squarespace',
 'Banyan Branch',
 'Banyan Branch',
 'Netbiscuits',
 'Gnodal',
 'Talenthouse',
 'Talenthouse']

In [22]:
NaN_lat_list = [40.7493756, 40.7208632, 40.720812, None, 40.7467544, 37.4499108, 37.4433618, 34.080633]
NaN_lon_list = [-73.9964352, -74.0035362, -74.0037199, None, -73.9953227, -122.1211875, -122.1630406, -118.3885556]
Nan_index_list = [2, 9, 14, 15, 16, 19, 20, 21]

In [23]:
my_dict = {key: value for key, value in zip(['name', 'latitude', 'longitude'], [NaN_name_list, NaN_lat_list, NaN_lon_list])}

In [24]:
my_dict['index'] = Nan_index_list

In [25]:
dict_df = pd.DataFrame(my_dict)
dict_df.set_index('index', inplace=True)

In [26]:
joined_df = df_subset.join(dict_df[['latitude', 'longitude']], on=df_subset.index)

In [27]:
df = df.join(joined_df[['latitude', 'longitude']], on=df.index)

In [28]:
df.loc[df['offices_latitude'].isna(), 'offices_latitude'] = df.loc[df['offices_latitude'].isna(), 'latitude']
df.loc[df['offices_longitude'].isna(), 'offices_longitude'] = df.loc[df['offices_longitude'].isna(), 'longitude']

In [29]:
df.drop(columns=['latitude', 'longitude'], inplace=True)

In [30]:
df.dropna(subset=['offices_latitude', 'offices_longitude'], how='all', inplace=True)
df = df.reset_index(drop=True)
df

,name,total_money_raised,offices_country_code,offices_state_code,offices_latitude,offices_longitude,offices_address_1,offices_address_2,offices_zip_code
0,Digg,$45M,USA,CA,37.764726,-122.394523,135 Mississippi St,NaN,94107
1,GENWI,$7.1M,USA,CA,33.817100,-111.903500,4966 El Camino Real,Suite 200,94022
2,Aviary,$19M,USA,NY,40.749376,-73.996435,243 West 30th Street,11th Floor,10001
3,Gilt Groupe,$236M,USA,NY,40.747270,-73.980064,2 Park Ave Fl 4,NaN,10016-5602
4,Sitemasher,C$5M,USA,FL,25.705057,-80.271378,NaN,NaN,NaN
5,99designs,$35M,USA,CA,37.795531,-122.400598,447 Battery St.,3rd Floor,94111
6,Shopflick,$8M,USA,CA,34.044939,-118.419067,10500 W. Pico Blvd,NaN,90064
7,deviantART,$13.5M,USA,CA,34.101578,-118.333340,7095 Hollywood Blvd #788,NaN,90028
8,Minted,$52.7M,USA,CA,37.797435,-122.403175,747 Front Street,2nd Floor,94111
9,Squarespace,$38.5M,USA,NY,40.720863,-74.003536,459 Broadway,5th Floor,10013


In [31]:
df.to_csv('data/df_mongo.csv', index=False)